### UFO Data
We have a file `ufo_data.tsv` of about 76Mb here. This is a big chunk of data to go through. Ideally we want to process items one at a time. The following block returns us a `generator` which we can iterate over and does exactly that.

In [91]:
import csv
def read_ufo_data():
    with open("ufo_data.tsv", 'r') as f:
        for row in csv.reader(f, delimiter="\t"):
            yield row
        

In [92]:
r = read_ufo_data()

In [93]:
r

<generator object read_ufo_data at 0x10ae54550>

Until now, we have only opened the file, we haven't even read the first line yet. This will only happen once we access the generator (by calling `next()` on it or iterating over it). These generator or iterator objects allow us to construct half-executed functions. That is called **lazy evulation** as they are only executed when it is needed. 

In [94]:
a = r.next()

In [95]:
a

['19951009',
 '19951009',
 ' Iowa City, IA',
 '',
 '',
 'Man repts. witnessing &quot;flash, followed by a classic UFO, w/ a tailfin at back.&quot; Red color on top half of tailfin. Became triangular.']

Now we have the first record of the file saved in the variable a. We have started **lazy evaluating** the code. To retrieve this record we did not have to load the entire content into memory and as soon as we release `a`, we will free this memory again. This can be very useful in various cases, like here when we are just interested to _find the first sighting in Australia_. **Excercise**: Can you write a short piece of code that finds that?

Remember: the place is listed at index `2` and a `"Australia" in` is totally sufficient here ;) .


In [101]:
filter(lambda x: "Australia" in x[2], read_ufo_data())[0]

['19970206',
 '19970212',
 ' Coober Pedy, S.A. (Approx. 250km south of) (Australia),',
 ' light',
 '',
 'SUMMARY:  A stormy night on the Stuart Highway. Sighting of a UFO which appeared to land,  a UFO chasing the car some 50m behind, and the sighting of a man which was extremely unusual looking.The first event to happen was the sighting of a &apos;roadwork&apos; man sweeping the highway.  Whilst driving along the highway, an extremely bright light suddenly appeared in front of us. At first we thought it was a truck coming towards us - common on the highway-but as we drove towards it we realised it was a stationary truck with a bright light attatched to the front of the grill. Upon getting closer to the light, it was that bright and blinding that we had to slow down to 10kms just to see the road and where we were going. Half the road was closed and suddenly we were able to see a man. There he was sweeping the road at 21:30(approx) 200kms from the nearest roadhouse. We slowly drove up t

## An iterative way
Although our iterator was doing lazy loading, `filter` generate an entire list, even though we only need the first sample. Obviously that is insufficient and we can do better in a more memory efficient manner. Introducing `itertools`. The [itertools](https://docs.python.org/2/library/itertools.html) library is preshipped with all (c)Python versions and contains iterative implementations of `map`, `filter` and such – appropriatly named `imap` and  `ifilter`. Lets use that one:

In [102]:
ifilter(lambda x: "Australia" in x[2], read_ufo_data())

Not only did that evaluate much quicker, as you can see, this creates another `iterator`, which means we are doing a lazy evaluation again. Let's use that to our advantage. To get the next (or in this case _first_) item, we can just call the `next()` function on it. Let's do that:

In [103]:
ifilter(lambda x: "Australia" in x[2], read_ufo_data()).next()

['19970206',
 '19970212',
 ' Coober Pedy, S.A. (Approx. 250km south of) (Australia),',
 ' light',
 '',
 'SUMMARY:  A stormy night on the Stuart Highway. Sighting of a UFO which appeared to land,  a UFO chasing the car some 50m behind, and the sighting of a man which was extremely unusual looking.The first event to happen was the sighting of a &apos;roadwork&apos; man sweeping the highway.  Whilst driving along the highway, an extremely bright light suddenly appeared in front of us. At first we thought it was a truck coming towards us - common on the highway-but as we drove towards it we realised it was a stationary truck with a bright light attatched to the front of the grill. Upon getting closer to the light, it was that bright and blinding that we had to slow down to 10kms just to see the road and where we were going. Half the road was closed and suddenly we were able to see a man. There he was sweeping the road at 21:30(approx) 200kms from the nearest roadhouse. We slowly drove up t

Well, that was much faster!

That is because we have only been reading the file – line by line – to the point that it was necessary. Even better with every line we checked, the reference was discarded and the memory cleared. This kept the footprint of memory very low throughout the entire execution. While also being much faster!

## Excercise

To do, one after another:

1. Can you write a low-memory solution, which returns the count of sightings per country (and for the US for each state)?

2. Can you make it stop when a certain total threshold has been met (given as a second parameter)?

3. Can you make it stop as soon as a c